<a href="https://colab.research.google.com/github/Ryuryu169/python_selflearning/blob/main/encrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA メモ

RSA の仕組み

公開鍵と秘密鍵の事前準備
1. 任意の素数を二つ p, q を定める
2. n = p × q とする
3. (p-1) * (q-1) と互いに素な数 e を任意に定める
4. e * d / n の余りが 1 となるような d を定める

* ここで、n と e が公開鍵、d は秘密鍵になる

* e はランダムな数値を出力し、ユークリッド法で余りが 1 のものを求める

* d は拡張ユークリッド法を用いることで求める事ができる

* 拡張ユークリッド法は ax + by = r となるような x と y を算出するために使う

暗号化

1. 送りたいメッセージ x に e 乗する
2. これを n で割った時の余りが暗号文になる

複号化

1. 暗号化されたメッセージ y に d 乗する
2. これを n で割った時の余りが元の文になる

In [ ]:
import random

p=int(input("Enter random number"))
q=int(input())

class rsa:
  def __init__(self,p,q):
    self.p = p
    self.q = q
    self.num = p * q
    self.less = (p-1) * (q-1)
    self.encryptKey = self.chooseE()
    self.decryptKey = self.remainOne()

  def checkGCM(self,i,j):
    remain = i % j
    while remain != 0:
      i = remain
      remain = j % i
      j = i

    return j

  def chooseE(self):
    remain = 0
    while remain != 1:
      encryptKey = random.randint(1,self.less)
      remain = self.checkGCM(self.less,encryptKey)
    return encryptKey #e

  def extgcd(self, a, b):
    if b:
      d, y, x = self.extgcd(b, a % b)
      y -= (a // b)*x
      return d, x, y
    return a, 1, 0

  def remainOne(self):
    remain, random, key = self.extgcd(self.less,self.encryptKey)
    if key < 0:
      key += self.less
    return key #d

  def returnRemain(self,a,b,n):
    tmp = 1
    for i in range(b):
      tmp = (a * tmp) % n
    return tmp

  def encrypt(self,msg):
    self.decryptKey = self.remainOne()
    encryptMsg = self.returnRemain(msg,self.encryptKey,self.num)
    print(encryptMsg)
    return encryptMsg

  def decrypt(self,encryptMsg):
    plainMessage = self.returnRemain(encryptMsg,self.decryptKey,self.num)
    return plainMessage

obj = rsa(p,q)
cypher = obj.encrypt()
msg = obj.decrypt(cypher)
print(msg)

91
83
2383
290


DH 法メモ

DH 法の仕組み

一連の処理
1. RSA などによって秘密鍵と任意の二つの数、素数 p と p > g となる自然数 g を用意する
2. p, g をそれぞれのユーザに送信
3. g を 秘密鍵で乗じて、p で余りを求める。
4. 3.で生成した余りを互いに交換する
5. 受け取った余りを秘密鍵で乗じて、pで余りを求める
6. 5.の余りが共通鍵になる。

実装方法
1. rsa 暗号をそれぞれで作る
2. p, g をクラス内で作成(0の時は作成、1の時は参照)
3. 互いに交換して確認

素数生成
1. ウィルソンの公式を用いて求める
2. g(n)≡2(n!) mod (n+1)かつ0≦g(n)≦nを満たすような数を求める
3. f(n) = g(n) + 2としたとき、素数となる

In [ ]:
import random
import math

p1=int(input("Enter your public key1 : "))
p2=int(input("Enter your public key2 : "))

class dh:
  def __init__(self,mode,p,g,key):
    if mode == 0:
      self.p, self.g = self.generate()
      self.key = key
    else:
      self.p = p
      self.g = g
      self.key = key

  def generate(self):
    while True:
      n = random.randint(1,100)
      if n % 2 == 1:
        break

    for i in range(n):
      tmp = 2 * math.factorial(n)
      if tmp % (n+1) == n:
        break

    return n, random.randint(1,n)

  def remainder(self,p,g,key):
    tmp = 1
    for i in range(key):
      tmp = (g * tmp) % p
    return tmp

def checkGCM(i,j):
    remain = i % j
    while remain != 0:
      i = remain
      remain = j % i
      j = i

    return j

dh1 = dh(0,0,0,p1)
dh2 = dh(1,dh1.p,dh1.g,p2)

send1 = dh1.remainder(dh1.p,dh1.g,dh1.key)
send2 = dh2.remainder(dh2.p,dh2.g,dh2.key)

message1 = dh1.remainder(dh1.p,send2,dh1.key)
message2 = dh2.remainder(dh2.p,send1,dh2.key)

print(send1,send2)
print(message1,message2)

9
7
6 16
11 11
